In [1]:
import sys
import numpy as np
import pickle
import py3Dmol
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed, GridspecLayout, Output

In [2]:
def plot_plddt_legend():
  thresh = ['plDDT:','Very low (<50)','Low (60)','OK (70)','Confident (80)','Very high (>90)']
  plt.figure(figsize=(1,0.1),dpi=100)
  for c in ["#FFFFFF","#FF0000","#FFFF00","#00FF00","#00FFFF","#0000FF"]:
    plt.bar(0, 0, color=c)
  plt.legend(thresh, frameon=False,
             loc='center', ncol=6,
             handletextpad=1,
             columnspacing=1,
             markerscale=0.5,)
  plt.axis(False)
  return plt

In [5]:
def plot_confidence(model_num=1, homooligomer=1, ptm=True):
  if ptm:
      pkl_file = f"result_model_{model_num}_ptm.pkl"
  else:
      pkl_file = f"result_model_{model_num}.pkl"
  with open(pkl_file, 'rb') as mpkl:
    resp = pickle.load(mpkl)
  msa_file = 'msas/bfd_uniclust_hits.a3m'
  row = 0
  with open(msa_file, 'r') as m:
    for line in m:
        row += 1
        if  row == 2:
            query_sequence = line.strip()
            break
 
  plt.figure(figsize=(10,3),dpi=100)
  plt.subplot(1,2,1); plt.title('Predicted lDDT')
  plt.plot(resp["plddt"])
  for n in range(homooligomer+1):
    x = n*(len(query_sequence))
    plt.plot([x,x],[0,100],color="black")
  plt.ylabel('plDDT')
  plt.xlabel('position')
  plt.subplot(1,2,2);plt.title('Predicted Aligned Error')
  plt.imshow(resp["predicted_aligned_error"], cmap="bwr",vmin=0,vmax=30)
  plt.colorbar()
  plt.xlabel('Scored residue')
  plt.ylabel('Aligned residue')
  return plt

In [6]:
def show_pdb(model_num=1, color="lDDT", amber_used=True, ptm=True,
             show_sidechains=False, show_mainchains=False, homooligomer=1):
  model_name = f"model_{model_num}"
  if amber_used:
    if ptm:
      pdb_filename = f"relaxed_{model_name}_ptm.pdb"
    else:
      pdb_filename = f"relaxed_{model_name}.pdb"
  else:
    if ptm:
      pdb_filename = f"unrelaxed_{model_name}_ptm.pdb"
    else:
      pdb_filename = f"unrelaxed_{model_name}.pdb"

  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_filename,'r').read(),'pdb')

  if color == "lDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    for n,chain,color in zip(range(homooligomer),list("ABCDEFGH"),
                     ["lime","cyan","magenta","yellow","salmon","white","blue","orange"]):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  view.zoomTo()

  if color == "lDDT": plot_plddt_legend().show()  
  
  plot_confidence(model_num, homooligomer).show()

  return view


In [7]:
interact(show_pdb, model_num=(1,5,1), color=["lDDT", "rainbow", "chain"], amber_used=False)

interactive(children=(IntSlider(value=1, description='model_num', max=5, min=1), Dropdown(description='color',…

<function __main__.show_pdb(model_num=1, color='lDDT', amber_used=True, ptm=True, show_sidechains=False, show_mainchains=False, homooligomer=1)>